## Option 2: Data Enrichment at-scale

### Setup

##### Create a dataset for storing the AI tables and another for storing the AI models

In [ ]:
%%bigquery
CREATE SCHEMA `team1-final`.fp_disney_stg_ai

Executing query with job ID: 85156562-aa9d-49de-b6aa-d7bbd8d2d682
Query executing: 0.42s


ERROR:
 409 Already Exists: Dataset team1-final:fp_disney_stg_ai

Location: US
Job ID: 85156562-aa9d-49de-b6aa-d7bbd8d2d682



In [ ]:
%%bigquery
CREATE SCHEMA `team1-final`.remote_models

Executing query with job ID: 4354153b-1a8e-41bb-b729-f6cb9751cd67
Query executing: 0.32s


ERROR:
 409 Already Exists: Dataset team1-final:remote_models

Location: US
Job ID: 4354153b-1a8e-41bb-b729-f6cb9751cd67



In [ ]:
%%bigquery
CREATE SCHEMA `team1-final`.remote_models_central

Query is running:   0%|          |

""


##### Before running this cell, create the remote connection and assign the IAM role `Vertex AI User` to the service account associated with the connection.

In [ ]:
%%bigquery
create or replace model remote_models.gemini_pro
  remote with connection `projects/team1-final/locations/us/connections/vertex_connection`
  options (endpoint = 'gemini-pro');

Query is running:   0%|          |

""


In [ ]:
%%bigquery
create or replace model remote_models_central.gemini_pro1
  remote with connection `projects/team1-final/locations/us-central1/connections/vertex_connection1`
  options (endpoint = 'gemini-pro');

Query is running:   0%|          |

""


# The seven columns being enriched are mpaa_rating from movies_total_gross and release_date, budget, box_office, imdb, metascore, rotten_tomatoes from disney_movies

### Part 1: Enrich Columns

#### mpaa_rating

In [ ]:
%%bigquery
select * except (data_source, load_time)
from disney_stg.Movies_Total_Gross
where mpaa_rating is not null
limit 5

Query is running:   0%|          |

Downloading:   0%|          |

,item_id,movie_title,release_date,genre,mpaa_rating,total_gross,inflation_adjusted_gross
0,Cinderella (15-Feb-50),Cinderella,15-Feb-50,Drama,G,"$85,000,000","$920,608,730"
1,The Straight Story (15-Oct-99),The Straight Story,15-Oct-99,Drama,G,"$6,197,866","$10,111,144"
2,Hannah Montana the Movie (10-Apr-09),Hannah Montana the Movie,10-Apr-09,Drama,G,"$79,576,189","$89,443,640"
3,Lady and the Tramp (22-Jun-55),Lady and the Tramp,22-Jun-55,Drama,G,"$93,600,000","$1,236,035,515"
4,The Rookie (29-Mar-02),The Rookie,29-Mar-02,Drama,G,"$75,600,072","$109,691,666"


#### Test the generate_text function

In [ ]:
%%bigquery
declare prompt_query STRING default "Given a dataset containing information about box office sales and ratings for movies, some entries are missing the mpaa_rating field. Your task is to predict the missing MPAA ratings based on your knowledge of the movie and your knowledge of the standard MPAA ratings such as PG and PG-13. Use this information to infer the likely MPAA rating, which classifies films to indicate suitability for certain audiences based on their content. Only return the actual rating, no reasoning. For example, 'G', 'PG, 'PG-13'. Return the output as JSON, include the item_id in the output.";

create or replace table fp_disney_stg_ai.mpaa_rating1 as
  select *
  from ML.generate_text(
    model remote_models.gemini_pro,
    (select concat(prompt_query, to_json_string(json_object("item_id", item_id, "movie_title", movie_title,
"release_date", release_date, "genre", genre,
"mpaa_rating", mpaa_rating, "total_gross", total_gross,
"inflation_adjusted_gross", inflation_adjusted_gross))) as prompt
    from disney_stg.Movies_Total_Gross
    where mpaa_rating is NULL
    order by item_id #limit 50
  ),
  struct(0 as temperature, 8192 as max_output_tokens, 0.0 as top_p, 1 as top_k, TRUE as flatten_json_output)
);


Query is running:   0%|          |

""


#### Tweak the prompt and save the output
##### [More details](https://cloud.google.com/bigquery/docs/generate-text#generate_text_from_text_data_by_using_a_prompt_from_a_query) on `ML.generate_text` parameters

In [ ]:
%%bigquery
select ml_generate_text_llm_result
from fp_disney_stg_ai.mpaa_rating1

Query is running:   0%|          |

Downloading:   0%|          |

,ml_generate_text_llm_result
0,"{""genre"":""Comedy"",""inflation_adjusted_gross"":""..."
1,"{""genre"":""Drama"",""inflation_adjusted_gross"":""$..."
2,"{""genre"":""Comedy"",""inflation_adjusted_gross"":""..."
3,"{""genre"":""Action"",""inflation_adjusted_gross"":""..."
4,"{""genre"":""Comedy"",""inflation_adjusted_gross"":""..."
5,"{""genre"":""Comedy"",""inflation_adjusted_gross"":""..."


#### Format the output to proper json

In [ ]:
%%bigquery
select json_value(ml_generate_text_llm_result, '$.item_id') as item_id,
  json_value(ml_generate_text_llm_result, '$.mpaa_rating') as mpaa_rating
from fp_disney_stg_ai.mpaa_rating1

Query is running:   0%|          |

Downloading:   0%|          |

,item_id,mpaa_rating
0,Where the Heart Is (23-Feb-90),PG
1,Wild Hearts Can't Be Broken (24-May-91),PG
2,Tin Men (6-Mar-87),PG
3,Tron (9-Jul-82),PG
4,Tough Guys (3-Oct-86),PG-13
5,Trenchcoat (11-Mar-83),PG




#### Update the disney_movies records with the predicted country

In [ ]:
%%bigquery
update disney_stg.Movies_Total_Gross
  set data_source = 'mpaa_rating_ai' where mpaa_rating is null

Query is running:   0%|          |

""


In [ ]:
%%bigquery
update disney_stg.Movies_Total_Gross set mpaa_rating =
  (select json_value(ml_generate_text_llm_result, '$.mpaa_rating')
   from fp_disney_stg_ai.mpaa_rating1
   where item_id = json_value(ml_generate_text_llm_result, '$.item_id'))
where mpaa_rating is NULL

Query is running:   0%|          |

""


#### Inspect the output

In [ ]:
%%bigquery
select * except(data_source, load_time)
from disney_stg.Movies_Total_Gross
#where country is not null

Query is running:   0%|          |

Downloading:   0%|          |

,item_id,movie_title,release_date,genre,mpaa_rating,total_gross,inflation_adjusted_gross
0,The Many Adventures of Winnie the Pooh (11-Mar...,The Many Adventures of Winnie the Pooh,11-Mar-77,None,G,$0,$0
1,Herbie Goes to Monte Carlo (24-Jun-77),Herbie Goes to Monte Carlo,24-Jun-77,None,G,"$28,000,000","$105,847,527"
2,Lady and the Tramp (22-Jun-55),Lady and the Tramp,22-Jun-55,Drama,G,"$93,600,000","$1,236,035,515"
3,The Rookie (29-Mar-02),The Rookie,29-Mar-02,Drama,G,"$75,600,072","$109,691,666"
4,Sleeping Beauty (29-Jan-59),Sleeping Beauty,29-Jan-59,Drama,G,"$9,464,608","$21,505,832"
...,...,...,...,...,...,...,...
574,The Invisible (27-Apr-07),The Invisible,27-Apr-07,Thriller/Suspense,PG-13,"$20,568,319","$25,202,168"
575,Unbreakable (22-Nov-00),Unbreakable,22-Nov-00,Thriller/Suspense,PG-13,"$94,999,143","$148,208,901"
576,Bon Voyage! (17-May-62),Bon Voyage!,17-May-62,Comedy,Not Rated,"$9,230,769","$109,581,646"
577,The Jungle Book (18-Oct-67),The Jungle Book,18-Oct-67,Musical,Not Rated,"$141,843,000","$789,612,346"


In [ ]:
%%bigquery
select count(*) from disney_stg.Movies_Total_Gross
where mpaa_rating is NULL;

Query is running:   0%|          |

Downloading:   0%|          |

,f0_
0,0


#### imdb

In [27]:
%%bigquery
select * except (data_source, load_time)
from disney_stg.Disney_Movies
where imdb is not null
limit 5

Query is running:   0%|          |

Downloading:   0%|          |

,item_id,title,director,cast,country,release_year,release_date,rating,movie_runtime,genre1,...,primary_production_company,additional_production_companies,produced_by,based_on,music_by,distributed_by,cinematography,edited_by,screenplay_by,date_added
0,Winnie the Pooh: A Very Merry Pooh Year (2002),Winnie the Pooh: A Very Merry Pooh Year,"Ed Wexler, Gary Katona, Jamie Mitchell","Jim Cummings, Peter Cullen, John Fiedler, Mich...",United States,2002,None,G,67 min,Animation,...,None,None,None,None,None,None,None,None,None,12-Nov-19
1,Winnie the Pooh: Springtime with Roo (2004),Winnie the Pooh: Springtime with Roo,"Elliot Bour, Saul Blinkoff","Jim Cummings, Ken Sansom, Jimmy Bennett, David...",United States,2004,None,G,70 min,Animation,...,None,None,None,None,None,None,None,None,None,12-Nov-19
2,Monsters University (2013),Monsters University,Dan Scanlon,"Billy Crystal, John Goodman, Steve Buscemi, He...",United States,2013,6/5/2013,G,104 min,Animation,...,None,None,Kori Rae,None,Randy Newman,"['Walt Disney Studios', 'Motion Pictures']","['Matt Aspbury', 'Jean-Claude Kalache']",Greg Snyder,None,12-Nov-19
3,Emil and the Detectives (1964),Emil and the Detectives,Peter Tewksbury,"Walter Slezak, Roger Mobley, Heinz Schubert, P...",United States,1964,12/18/1964,TV-PG,98 min,Action-Adventure,...,Walt Disney Productions,None,"['John McKimson', 'Walt Disney']","['Emil and the Detectives', 'by', 'Erich Kästn...",None,Buena Vista Distribution,None,Thomas Stanford,None,12-Nov-19
4,The Little Mermaid II: Return to the Sea (2000),The Little Mermaid II: Return to the Sea,Jim Kammerud,"Jodi Benson, Samuel Wright, Tara Charendoff, P...","United States, Canada, Australia, Taiwan",2000,None,G,77 min,Animation,...,None,None,None,None,None,None,None,None,None,12-Nov-19


In [28]:
%%bigquery
declare prompt_query STRING default "Based on the provided information about the movie and what you know about the movie, predict the IMDB score of the movie. For example, 7.0, 8.0, 4.0, 6.0, etc. Predict a value even if you are unsure. Return the output as json, include the item_id in the output as well";

create or replace table fp_disney_stg_ai.imdb1 as
  select *
  from ML.generate_text(
    model remote_models.gemini_pro,
    (
    select concat(prompt_query, to_json_string(json_object("item_id", dm.item_id, "title", dm.title,
"director", dm.director, "cast", dm.cast,
"country", dm.country, "release_year", dm.release_year,
"release_date", dm.release_date, "rating", dm.rating,
"movie_runtime", dm.movie_runtime, "genre1", dm.genre1,
"genre2", dm.genre2, "genre3", dm.genre3,
"description", dm.description, "budget", dm.budget,
"box_office", dm.box_office, "imdb", dm.imdb,
"metascore", dm.metascore, "rotten_tomatoes", dm.rotten_tomatoes,
"language", dm.language, "primary_production_company", dm.primary_production_company,
"additional_production_companies", dm.additional_production_companies, "produced_by", dm.produced_by,
"based_on", dm.based_on, "music_by", dm.music_by,
"distributed_by", dm.distributed_by, "cinematography", dm.cinematography,
"edited_by", dm.edited_by, "screenplay_by", dm.screenplay_by,
"date_added", dm.date_added))) as prompt
    from disney_stg.Disney_Movies as dm
    where imdb is null
    order by item_id
  ),
  struct(0 as temperature, 8192 as max_output_tokens, 0.0 as top_p, 1 as top_k, TRUE as flatten_json_output)
);


Query is running:   0%|          |

""


In [29]:
%%bigquery
select ml_generate_text_llm_result
from fp_disney_stg_ai.imdb1

Query is running:   0%|          |

Downloading:   0%|          |

,ml_generate_text_llm_result
0,"```json\n{""additional_production_companies"":""W..."
1,"```json\n{""additional_production_companies"":nu..."
2,"Based on the information provided, here is my ..."
3,"```json\n{""additional_production_companies"":nu..."
4,## IMDB Score Prediction\n\nBased on the infor...
...,...
979,"## Predicting IMDB score for ""The Cat from Out..."
980,"```json\n{""additional_production_companies"":nu..."
981,"```json\n{""additional_production_companies"":nu..."
982,"```json\n{""additional_production_companies"":nu..."


In [30]:
%%bigquery
select json_value(ml_generate_text_llm_result, '$.item_id') as item_id,
  json_value(ml_generate_text_llm_result, '$.imdb') as imdb
from fp_disney_stg_ai.imdb1

Query is running:   0%|          |

Downloading:   0%|          |

,item_id,imdb
0,None,None
1,None,None
2,None,None
3,None,None
4,None,None
...,...,...
979,None,None
980,None,None
981,None,None
982,None,None


In [31]:
%%bigquery
update disney_stg.Disney_Movies
  set data_source = 'imdb_ai' where imdb is null

Query is running:   0%|          |

""


In [32]:
%%bigquery
UPDATE disney_stg.Disney_Movies
SET imdb = CAST(
    (SELECT json_value(ml_generate_text_llm_result, '$.imdb')
     FROM fp_disney_stg_ai.imdb1
     WHERE item_id = json_value(ml_generate_text_llm_result, '$.item_id')) AS FLOAT64
)
WHERE imdb IS NULL;

Query is running:   0%|          |

""


In [33]:
%%bigquery
select * except(data_source, load_time)
from disney_stg.Disney_Movies
where imdb is not null

Query is running:   0%|          |

Downloading:   0%|          |

,item_id,title,director,cast,country,release_year,release_date,rating,movie_runtime,genre1,...,primary_production_company,additional_production_companies,produced_by,based_on,music_by,distributed_by,cinematography,edited_by,screenplay_by,date_added
0,Winnie the Pooh: Springtime with Roo (2004),Winnie the Pooh: Springtime with Roo,"Elliot Bour, Saul Blinkoff","Jim Cummings, Ken Sansom, Jimmy Bennett, David...",United States,2004,None,G,70 min,Animation,...,None,None,None,None,None,None,None,None,None,12-Nov-19
1,Winnie the Pooh: A Very Merry Pooh Year (2002),Winnie the Pooh: A Very Merry Pooh Year,"Ed Wexler, Gary Katona, Jamie Mitchell","Jim Cummings, Peter Cullen, John Fiedler, Mich...",United States,2002,None,G,67 min,Animation,...,None,None,None,None,None,None,None,None,None,12-Nov-19
2,Monsters University (2013),Monsters University,Dan Scanlon,"Billy Crystal, John Goodman, Steve Buscemi, He...",United States,2013,6/5/2013,G,104 min,Animation,...,None,None,Kori Rae,None,Randy Newman,"['Walt Disney Studios', 'Motion Pictures']","['Matt Aspbury', 'Jean-Claude Kalache']",Greg Snyder,None,12-Nov-19
3,Emil and the Detectives (1964),Emil and the Detectives,Peter Tewksbury,"Walter Slezak, Roger Mobley, Heinz Schubert, P...",United States,1964,12/18/1964,TV-PG,98 min,Action-Adventure,...,Walt Disney Productions,None,"['John McKimson', 'Walt Disney']","['Emil and the Detectives', 'by', 'Erich Kästn...",None,Buena Vista Distribution,None,Thomas Stanford,None,12-Nov-19
4,The Little Mermaid II: Return to the Sea (2000),The Little Mermaid II: Return to the Sea,Jim Kammerud,"Jodi Benson, Samuel Wright, Tara Charendoff, P...","United States, Canada, Australia, Taiwan",2000,None,G,77 min,Animation,...,None,None,None,None,None,None,None,None,None,12-Nov-19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,The Black Hole (1979),The Black Hole,Gary Nelson,"Maximilian Schell, Anthony Perkins, Robert For...",United States,1979,12/18/1979,PG,98 min,Action-Adventure,...,Walt Disney Productions,None,Ron Miller,None,John Barry,Buena Vista Distribution,Frank Phillips,Gregg McLaughlin,"['Gerry Day', 'Jeb Rosebrook']",12-Nov-19
70,Togo (2019),Togo,Ericson Core,"Willem Dafoe, Julianne Nicholson, Christopher ...",United States,2019,12/20/2019,PG,117 min,Action-Adventure,...,Walt Disney Pictures,None,Kim Zubick,None,Mark Isham,Disney+,Ericson Core,Martin Pensa,None,20-Dec-19
71,Homeward Bound II: Lost in San Francisco (1996),Homeward Bound II: Lost in San Francisco,David Ellis,"Robert Hays, Kim Greist, Veronica Lauren, Kevi...",United States,1996,3/8/1996,G,90 min,Action-Adventure,...,Walt Disney Pictures,None,Barry Jossen,"['Characters', 'by', 'Sheila Burnford']",Bruce Broughton,Buena Vista Pictures,Jack Conroy,"['Peter E. Berger', 'Michael A. Stevenson']",None,1-May-20
72,Homeward Bound: The Incredible Journey (1993),Homeward Bound: The Incredible Journey,Duwayne Dunham,"Robert Hays, Kim Greist, Jean Smart, Veronica ...",United States,1993,2/3/1993,G,85 min,Action-Adventure,...,Walt Disney Pictures,None,"['Jeffrey Chernov', 'Franklin R. Levy']","['The Incredible Journey', 'by', 'Sheila Burnf...",Bruce Broughton,Buena Vista Pictures,Reed Smoot,Jonathan P. Shaw,None,1-May-20


In [34]:
%%bigquery
select count(*) from disney_stg.Disney_Movies
where imdb is NULL;

Query is running:   0%|          |

Downloading:   0%|          |

,f0_
0,978


#### metascore

In [35]:
%%bigquery
select * except (data_source, load_time)
from disney_stg.Disney_Movies
where metascore is not null
limit 5

Query is running:   0%|          |

Downloading:   0%|          |

,item_id,title,director,cast,country,release_year,release_date,rating,movie_runtime,genre1,...,primary_production_company,additional_production_companies,produced_by,based_on,music_by,distributed_by,cinematography,edited_by,screenplay_by,date_added
0,"Monsters, Inc. (2001)","Monsters, Inc.",Pete Docter,"John Goodman, Billy Crystal, Mary Gibbs, Steve...",United States,2001,10/28/2001,G,93 min,Animation,...,Walt Disney Pictures,Pixar Animation Studios,Darla K. Anderson,None,Randy Newman,Buena Vista Pictures Distribution,None,"['Robert Grahamjones', 'Jim Stewart']","['Andrew Stanton', 'Dan Gerson']",12-Nov-19
1,Monsters University (2013),Monsters University,Dan Scanlon,"Billy Crystal, John Goodman, Steve Buscemi, He...",United States,2013,6/5/2013,G,104 min,Animation,...,None,None,Kori Rae,None,Randy Newman,"['Walt Disney Studios', 'Motion Pictures']","['Matt Aspbury', 'Jean-Claude Kalache']",Greg Snyder,None,12-Nov-19
2,The Computer Wore Tennis Shoes (1969),The Computer Wore Tennis Shoes,Robert Butler,"Kurt Russell, Cesar Romero, Joe Flynn, William...",United States,1969,12/31/1969,G,90 min,Comedy,...,Walt Disney Productions,None,Bill Anderson,None,Robert F. Brunner,Buena Vista Distribution,Frank V. Phillips,Cotton Warburton,None,12-Nov-19
3,Artemis Fowl (2020),Artemis Fowl,Kenneth Branagh,None,United States,2020,6/12/2020,PG,96 min,Action-Adventure,...,Walt Disney Pictures,"TriBeCa Productions, Marzano Films","['Kenneth Branagh', 'Judy Hofflund']","['Artemis Fowl', 'by', 'Eoin Colfer']",Patrick Doyle,Walt Disney Studios Motion Pictures,Haris Zambarloukos,Matthew Tucker,"['Conor McPherson', 'Hamish McColl']",12-Jun-20
4,Safety (2020),Safety,Reginald Hudlin,"Jay Reeves, Thaddeus J. Mixson, Corinne Foxx, ...",United States,2020,12/11/2020,PG,123 min,Biographical,...,Walt Disney Pictures,"Mayhem Pictures, Select Films","['Mark Ciardi', 'Gordon Gray']",None,Marcus Miller,Disney+,Shane Hurlbut,Terel Gibson,None,11-Dec-20


In [36]:
%%bigquery
declare prompt_query STRING default "Based on the provided information about the movie and what you know about the movie, predict the metascore of the movie. For example, 74, 30, 48, 59, etc. Predict a value even if you are unsure. Return the output as json, include the item_id in the output as well";

create or replace table fp_disney_stg_ai.metascore1 as
  select *
  from ML.generate_text(
    model remote_models.gemini_pro,
    (
    select concat(prompt_query, to_json_string(json_object("item_id", dm.item_id, "title", dm.title,
"director", dm.director, "cast", dm.cast,
"country", dm.country, "release_year", dm.release_year,
"release_date", dm.release_date, "rating", dm.rating,
"movie_runtime", dm.movie_runtime, "genre1", dm.genre1,
"genre2", dm.genre2, "genre3", dm.genre3,
"description", dm.description, "budget", dm.budget,
"box_office", dm.box_office, "imdb", dm.imdb,
"metascore", dm.metascore, "rotten_tomatoes", dm.rotten_tomatoes,
"language", dm.language, "primary_production_company", dm.primary_production_company,
"additional_production_companies", dm.additional_production_companies, "produced_by", dm.produced_by,
"based_on", dm.based_on, "music_by", dm.music_by,
"distributed_by", dm.distributed_by, "cinematography", dm.cinematography,
"edited_by", dm.edited_by, "screenplay_by", dm.screenplay_by,
"date_added", dm.date_added))) as prompt
    from disney_stg.Disney_Movies as dm
    where metascore is null
    order by item_id
  ),
  struct(0 as temperature, 8192 as max_output_tokens, 0.0 as top_p, 1 as top_k, TRUE as flatten_json_output)
);


Query is running:   0%|          |

""


In [37]:
%%bigquery
select ml_generate_text_llm_result
from fp_disney_stg_ai.metascore1

Query is running:   0%|          |

Downloading:   0%|          |

,ml_generate_text_llm_result
0,"```json\n{""additional_production_companies"":nu..."
1,"```json\n{""additional_production_companies"":nu..."
2,"```json\n{""additional_production_companies"":nu..."
3,"```json\n{""additional_production_companies"":nu..."
4,"```json\n{""additional_production_companies"":nu..."
...,...
822,"```json\n{""additional_production_companies"":nu..."
823,"```json\n{""additional_production_companies"":nu..."
824,"```json\n{""additional_production_companies"":nu..."
825,"```json\n{""additional_production_companies"":nu..."


In [38]:
%%bigquery
select json_value(ml_generate_text_llm_result, '$.item_id') as item_id,
  json_value(ml_generate_text_llm_result, '$.metascore') as metascore
from fp_disney_stg_ai.metascore1

Query is running:   0%|          |

Downloading:   0%|          |

,item_id,metascore
0,None,None
1,None,None
2,None,None
3,None,None
4,None,None
...,...,...
822,None,None
823,None,None
824,None,None
825,None,None


In [39]:
%%bigquery
update disney_stg.Disney_Movies
  set data_source = 'metascore_ai' where metascore is null

Query is running:   0%|          |

""


In [40]:
%%bigquery
UPDATE disney_stg.Disney_Movies
SET metascore = CAST(
    (SELECT json_value(ml_generate_text_llm_result, '$.metascore')
     FROM fp_disney_stg_ai.metascore1
     WHERE item_id = json_value(ml_generate_text_llm_result, '$.item_id')) AS INT64
)
WHERE metascore IS NULL;

Query is running:   0%|          |

""


In [41]:
%%bigquery
select * except(data_source, load_time)
from disney_stg.Disney_Movies
where metascore is not null

Query is running:   0%|          |

Downloading:   0%|          |

,item_id,title,director,cast,country,release_year,release_date,rating,movie_runtime,genre1,...,primary_production_company,additional_production_companies,produced_by,based_on,music_by,distributed_by,cinematography,edited_by,screenplay_by,date_added
0,National Treasure (2004),National Treasure,Jon Turteltaub,"Nicolas Cage, Jon Voight, Harvey Keitel, Diane...",United States,2004,11/19/2004,PG,132 min,Action-Adventure,...,Walt Disney Pictures,"Jerry Bruckheimer Films, Junction Entertainmen...","['Jerry Bruckheimer', 'Jon Turteltaub']",None,Trevor Rabin,Buena Vista Pictures,Caleb Deschanel,William Goldenberg,"['Jim Kouf', 'Cormac Wibberley', 'Marianne Wib...",30-Apr-20
1,Melody Time (1948),Melody Time,"Clyde Geronimi, Jack Kinney, Hamilton Luske","Roy Rogers, Trigger , Dennis Day, The Andrews ...",United States,1948,5/27/1948,TV-PG,76 min,Animation,...,Walt Disney Productions,None,Walt Disney,None,"['Eliot Daniel', 'Paul J. Smith', 'Ken Darby']","RKO Radio Pictures, Inc.",None,"['Donald Halliday', 'Thomas Scott']",None,12-Nov-19
2,National Treasure: Book of Secrets (2007),National Treasure: Book of Secrets,Jon Turteltaub,"Nicolas Cage, Jon Voight, Harvey Keitel, Ed Ha...",United States,2007,12/21/2007,PG,125 min,Action-Adventure,...,Walt Disney Pictures,"Jerry Bruckheimer Films, Junction Entertainmen...","['Jerry Bruckheimer', 'Jon Turteltaub']","['Characters', 'by', 'Jim Kouf', 'Oren Aviv', ...",Trevor Rabin,Walt Disney Studios Motion Pictures,"['John Schwartzman', 'Amir Mokri']","['William Goldenberg', 'David Rennie']","['Cormac Wibberley', 'Marianne Wibberley']",12-Nov-19
3,The Finest Hours (2016),The Finest Hours,Craig Gillespie,"Chris Pine, Casey Affleck, Ben Foster, Hollida...",United States,2016,1/25/2016,PG-13,119 min,Action-Adventure,...,Walt Disney Pictures,"Whitaker Entertainment, Red Hawk Entertainment","['Jim Whitaker', 'Dorothy Aufiero']","[""The Finest Hours: The True Story of the U.S....",Carter Burwell,"['Walt Disney Studios', 'Motion Pictures']",Javier Aguirresarobe,Tatiana S. Riegel,"['Scott Silver', 'Paul Tamasy', 'Eric Johnson']",6-Mar-20
4,"Monsters, Inc. (2001)","Monsters, Inc.",Pete Docter,"John Goodman, Billy Crystal, Mary Gibbs, Steve...",United States,2001,10/28/2001,G,93 min,Animation,...,Walt Disney Pictures,Pixar Animation Studios,Darla K. Anderson,None,Randy Newman,Buena Vista Pictures Distribution,None,"['Robert Grahamjones', 'Jim Stewart']","['Andrew Stanton', 'Dan Gerson']",12-Nov-19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,102 Dalmatians (2000),102 Dalmatians,Kevin Lima,"Glenn Close, Ioan Gruffudd, Alice Evans, Tim M...","United States, United Kingdom",2000,11/22/2000,G,104 min,Action-Adventure,...,Walt Disney Pictures,"Cruella Productions, Kanzaman S.A.M. Films",Edward S. Feldman,None,David Newman,Buena Vista Pictures,Adrian Biddle,Gregory Perler,"['Kristen Buckley', 'Brian Regan', 'Bob Tzudik...",12-Nov-19
227,White Fang (1991),White Fang,Randal Kleiser,"Klaus Brandauer, Ethan Hawke, Seymour Cassel, ...",United States,1991,1/18/1991,PG,109 min,Action-Adventure,...,Walt Disney Pictures,Silver Screen Partners IV,Markay Powell,"['White Fang', 'by', 'Jack London']","['Basil Poledouris', 'Hans Zimmer', '(addition...",Buena Vista Pictures,Tony Pierce-Roberts,Lisa Day,"['Jeanne Rosenberg', 'Nick Thiel', 'David Fall...",12-Nov-19
228,The Young Black Stallion (2003),The Young Black Stallion,Simon Wincer,"Richard Romanus, Biana Tamimi, Patrick Elyas, ...",United States,2003,12/25/2003,G,51 min,Action-Adventure,...,Walt Disney Pictures,The Kennedy/Marshall Company,"['Fred Roos', 'Frank Marshall']","['Walter Farley', 'Steven Farley']",William Ross,Buena Vista Pictures,Reed Smoot,"['Bud Smith', 'Terry Blythe', 'M. Scott Smith']",Jeanne Rosenberg,12-Nov-19
229,Benji the Hunted (1987),Benji the Hunted,Joe Camp,"Red Steagall, Frank Inn, Nancy Francis, Mike F...",United States,1987,6/5/1987,G,89 min,Action-Adventure,...,Walt Disney Pictures,"Mulberry Square Pro

In [42]:
%%bigquery
select count(*) from disney_stg.Disney_Movies
where metascore is NULL;

Query is running:   0%|          |

Downloading:   0%|          |

,f0_
0,821


#### rotten_tomatoes

In [43]:
%%bigquery
select * except (data_source, load_time)
from disney_stg.Disney_Movies
where rotten_tomatoes is not null
limit 5

Query is running:   0%|          |

Downloading:   0%|          |

,item_id,title,director,cast,country,release_year,release_date,rating,movie_runtime,genre1,...,primary_production_company,additional_production_companies,produced_by,based_on,music_by,distributed_by,cinematography,edited_by,screenplay_by,date_added
0,National Treasure (2004),National Treasure,Jon Turteltaub,"Nicolas Cage, Jon Voight, Harvey Keitel, Diane...",United States,2004,11/19/2004,PG,132 min,Action-Adventure,...,Walt Disney Pictures,"Jerry Bruckheimer Films, Junction Entertainmen...","['Jerry Bruckheimer', 'Jon Turteltaub']",None,Trevor Rabin,Buena Vista Pictures,Caleb Deschanel,William Goldenberg,"['Jim Kouf', 'Cormac Wibberley', 'Marianne Wib...",30-Apr-20
1,Tall Tale (1995),Tall Tale,Jeremiah Chechik,"Scott Glenn, Oliver Platt, Nick Stahl, Stephen...",United States,1995,3/24/1995,PG,97 min,Action-Adventure,...,Walt Disney Pictures,Caravan Pictures,"['Roger Birnbaum', 'Joe Roth']",None,Randy Edelman,Buena Vista Pictures,Janusz Kamiński,Richard Chew,None,12-Nov-19
2,Melody Time (1948),Melody Time,"Clyde Geronimi, Jack Kinney, Hamilton Luske","Roy Rogers, Trigger , Dennis Day, The Andrews ...",United States,1948,5/27/1948,TV-PG,76 min,Animation,...,Walt Disney Productions,None,Walt Disney,None,"['Eliot Daniel', 'Paul J. Smith', 'Ken Darby']","RKO Radio Pictures, Inc.",None,"['Donald Halliday', 'Thomas Scott']",None,12-Nov-19
3,National Treasure: Book of Secrets (2007),National Treasure: Book of Secrets,Jon Turteltaub,"Nicolas Cage, Jon Voight, Harvey Keitel, Ed Ha...",United States,2007,12/21/2007,PG,125 min,Action-Adventure,...,Walt Disney Pictures,"Jerry Bruckheimer Films, Junction Entertainmen...","['Jerry Bruckheimer', 'Jon Turteltaub']","['Characters', 'by', 'Jim Kouf', 'Oren Aviv', ...",Trevor Rabin,Walt Disney Studios Motion Pictures,"['John Schwartzman', 'Amir Mokri']","['William Goldenberg', 'David Rennie']","['Cormac Wibberley', 'Marianne Wibberley']",12-Nov-19
4,The Finest Hours (2016),The Finest Hours,Craig Gillespie,"Chris Pine, Casey Affleck, Ben Foster, Hollida...",United States,2016,1/25/2016,PG-13,119 min,Action-Adventure,...,Walt Disney Pictures,"Whitaker Entertainment, Red Hawk Entertainment","['Jim Whitaker', 'Dorothy Aufiero']","[""The Finest Hours: The True Story of the U.S....",Carter Burwell,"['Walt Disney Studios', 'Motion Pictures']",Javier Aguirresarobe,Tatiana S. Riegel,"['Scott Silver', 'Paul Tamasy', 'Eric Johnson']",6-Mar-20


In [44]:
%%bigquery
declare prompt_query STRING default "Based on the provided information about the movie and what you know about the movie, predict the rotten_tomatoes score of the movie. For example, 94%, 21%, 37%, 56%, etc. Predict a value even if you are unsure. Return the output as json, include the item_id in the output as well";

create or replace table fp_disney_stg_ai.rotten_tomatoes1 as
  select *
  from ML.generate_text(
    model remote_models.gemini_pro,
    (
    select concat(prompt_query, to_json_string(json_object("item_id", dm.item_id, "title", dm.title,
"director", dm.director, "cast", dm.cast,
"country", dm.country, "release_year", dm.release_year,
"release_date", dm.release_date, "rating", dm.rating,
"movie_runtime", dm.movie_runtime, "genre1", dm.genre1,
"genre2", dm.genre2, "genre3", dm.genre3,
"description", dm.description, "budget", dm.budget,
"box_office", dm.box_office, "imdb", dm.imdb,
"metascore", dm.metascore, "rotten_tomatoes", dm.rotten_tomatoes,
"language", dm.language, "primary_production_company", dm.primary_production_company,
"additional_production_companies", dm.additional_production_companies, "produced_by", dm.produced_by,
"based_on", dm.based_on, "music_by", dm.music_by,
"distributed_by", dm.distributed_by, "cinematography", dm.cinematography,
"edited_by", dm.edited_by, "screenplay_by", dm.screenplay_by,
"date_added", dm.date_added))) as prompt
    from disney_stg.Disney_Movies as dm
    where rotten_tomatoes is null
    order by item_id
  ),
  struct(0 as temperature, 8192 as max_output_tokens, 0.0 as top_p, 1 as top_k, TRUE as flatten_json_output)
);


Query is running:   0%|          |

""


In [45]:
%%bigquery
select ml_generate_text_llm_result
from fp_disney_stg_ai.rotten_tomatoes1

Query is running:   0%|          |

Downloading:   0%|          |

,ml_generate_text_llm_result
0,"```json\n{""additional_production_companies"":nu..."
1,"```json\n{""additional_production_companies"":nu..."
2,"```json\n{""additional_production_companies"":nu..."
3,"```json\n{""additional_production_companies"":nu..."
4,"```json\n{""additional_production_companies"":nu..."
...,...
741,"```json\n{""additional_production_companies"":nu..."
742,"```json\n{""additional_production_companies"":nu..."
743,"```json\n{""additional_production_companies"":nu..."
744,"```json\n{""additional_production_companies"":nu..."


In [46]:
%%bigquery
select json_value(ml_generate_text_llm_result, '$.item_id') as item_id,
  json_value(ml_generate_text_llm_result, '$.rotten_tomatoes') as rotten_tomatoes
from fp_disney_stg_ai.rotten_tomatoes1

Query is running:   0%|          |

Downloading:   0%|          |

,item_id,rotten_tomatoes
0,None,None
1,None,None
2,None,None
3,None,None
4,None,None
...,...,...
741,None,None
742,None,None
743,None,None
744,None,None


In [47]:
%%bigquery
update disney_stg.Disney_Movies
  set data_source = 'rotten_tomatoes_ai' where rotten_tomatoes is null

Query is running:   0%|          |

""


In [48]:
%%bigquery
update disney_stg.Disney_Movies set rotten_tomatoes =
  (select json_value(ml_generate_text_llm_result, '$.rotten_tomatoes')
   from fp_disney_stg_ai.rotten_tomatoes1
   where item_id = json_value(ml_generate_text_llm_result, '$.item_id'))
where rotten_tomatoes is NULL

Query is running:   0%|          |

""


In [49]:
%%bigquery
select * except(data_source, load_time)
from disney_stg.Disney_Movies
where rotten_tomatoes is not null

Query is running:   0%|          |

Downloading:   0%|          |

,item_id,title,director,cast,country,release_year,release_date,rating,movie_runtime,genre1,...,primary_production_company,additional_production_companies,produced_by,based_on,music_by,distributed_by,cinematography,edited_by,screenplay_by,date_added
0,National Treasure (2004),National Treasure,Jon Turteltaub,"Nicolas Cage, Jon Voight, Harvey Keitel, Diane...",United States,2004,11/19/2004,PG,132 min,Action-Adventure,...,Walt Disney Pictures,"Jerry Bruckheimer Films, Junction Entertainmen...","['Jerry Bruckheimer', 'Jon Turteltaub']",None,Trevor Rabin,Buena Vista Pictures,Caleb Deschanel,William Goldenberg,"['Jim Kouf', 'Cormac Wibberley', 'Marianne Wib...",30-Apr-20
1,Tall Tale (1995),Tall Tale,Jeremiah Chechik,"Scott Glenn, Oliver Platt, Nick Stahl, Stephen...",United States,1995,3/24/1995,PG,97 min,Action-Adventure,...,Walt Disney Pictures,Caravan Pictures,"['Roger Birnbaum', 'Joe Roth']",None,Randy Edelman,Buena Vista Pictures,Janusz Kamiński,Richard Chew,None,12-Nov-19
2,Melody Time (1948),Melody Time,"Clyde Geronimi, Jack Kinney, Hamilton Luske","Roy Rogers, Trigger , Dennis Day, The Andrews ...",United States,1948,5/27/1948,TV-PG,76 min,Animation,...,Walt Disney Productions,None,Walt Disney,None,"['Eliot Daniel', 'Paul J. Smith', 'Ken Darby']","RKO Radio Pictures, Inc.",None,"['Donald Halliday', 'Thomas Scott']",None,12-Nov-19
3,National Treasure: Book of Secrets (2007),National Treasure: Book of Secrets,Jon Turteltaub,"Nicolas Cage, Jon Voight, Harvey Keitel, Ed Ha...",United States,2007,12/21/2007,PG,125 min,Action-Adventure,...,Walt Disney Pictures,"Jerry Bruckheimer Films, Junction Entertainmen...","['Jerry Bruckheimer', 'Jon Turteltaub']","['Characters', 'by', 'Jim Kouf', 'Oren Aviv', ...",Trevor Rabin,Walt Disney Studios Motion Pictures,"['John Schwartzman', 'Amir Mokri']","['William Goldenberg', 'David Rennie']","['Cormac Wibberley', 'Marianne Wibberley']",12-Nov-19
4,The Finest Hours (2016),The Finest Hours,Craig Gillespie,"Chris Pine, Casey Affleck, Ben Foster, Hollida...",United States,2016,1/25/2016,PG-13,119 min,Action-Adventure,...,Walt Disney Pictures,"Whitaker Entertainment, Red Hawk Entertainment","['Jim Whitaker', 'Dorothy Aufiero']","[""The Finest Hours: The True Story of the U.S....",Carter Burwell,"['Walt Disney Studios', 'Motion Pictures']",Javier Aguirresarobe,Tatiana S. Riegel,"['Scott Silver', 'Paul Tamasy', 'Eric Johnson']",6-Mar-20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
314,Eight Below (2006),Eight Below,Frank Marshall,"Paul Walker, Bruce Greenwood, Moon Bloodgood, ...",United States,2006,2/17/2006,PG,122 min,Action-Adventure,...,Walt Disney Pictures,"Spyglass Entertainment, Mandeville Films, The ...","['Patrick Crowley', 'David Hoberman']","['Toshirô Ishidô', 'Koreyoshi Kurahara', 'Tats...",Mark Isham,Buena Vista Pictures,Don Burgess,Christopher Rouse,David DiGillio,12-Nov-19
315,102 Dalmatians (2000),102 Dalmatians,Kevin Lima,"Glenn Close, Ioan Gruffudd, Alice Evans, Tim M...","United States, United Kingdom",2000,11/22/2000,G,104 min,Action-Adventure,...,Walt Disney Pictures,"Cruella Productions, Kanzaman S.A.M. Films",Edward S. Feldman,None,David Newman,Buena Vista Pictures,Adrian Biddle,Gregory Perler,"['Kristen Buckley', 'Brian Regan', 'Bob Tzudik...",12-Nov-19
316,Aliens of the Deep (2005),Aliens of the Deep,"James Cameron, Steven Quale","Dr. Anatoly Sagalevitch, Genya Chernaiev, Vict...",United States,2005,1/28/2005,G,100 min,Action-Adventure,...,Walt Disney Pictures,"Walden Media, Earthship Productions","['James Cameron', 'Andrew Wight']",None,Jeehun Hwang,Buena Vista Pictures,"['James Cameron', 'Vince Pace', 'Ron Allum']","['Matthew Kregor', 'Fiona Wight']",None,12-Nov-19
317,The Incredible Journey (1963),The Incredible Journey,Fletcher Markle,"Émile Genest, John Drainie, Tommy Tweed, Sandr...","Canada, United States",1963,11/20/1963,G,80 min,Action-Adventure,...,Walt Disney Productions,None,"['James Algar', 'Walt Disney']","['The Incredib

In [50]:
%%bigquery
select count(*) from disney_stg.Disney_Movies
where rotten_tomatoes is NULL;

Query is running:   0%|          |

Downloading:   0%|          |

,f0_
0,733


#### release_date

In [51]:
%%bigquery
select * except (data_source, load_time)
from disney_stg.Disney_Movies
where release_date is not null
limit 5

Query is running:   0%|          |

Downloading:   0%|          |

,item_id,title,director,cast,country,release_year,release_date,rating,movie_runtime,genre1,...,primary_production_company,additional_production_companies,produced_by,based_on,music_by,distributed_by,cinematography,edited_by,screenplay_by,date_added
0,National Treasure (2004),National Treasure,Jon Turteltaub,"Nicolas Cage, Jon Voight, Harvey Keitel, Diane...",United States,2004,11/19/2004,PG,132 min,Action-Adventure,...,Walt Disney Pictures,"Jerry Bruckheimer Films, Junction Entertainmen...","['Jerry Bruckheimer', 'Jon Turteltaub']",None,Trevor Rabin,Buena Vista Pictures,Caleb Deschanel,William Goldenberg,"['Jim Kouf', 'Cormac Wibberley', 'Marianne Wib...",30-Apr-20
1,Tall Tale (1995),Tall Tale,Jeremiah Chechik,"Scott Glenn, Oliver Platt, Nick Stahl, Stephen...",United States,1995,3/24/1995,PG,97 min,Action-Adventure,...,Walt Disney Pictures,Caravan Pictures,"['Roger Birnbaum', 'Joe Roth']",None,Randy Edelman,Buena Vista Pictures,Janusz Kamiński,Richard Chew,None,12-Nov-19
2,Melody Time (1948),Melody Time,"Clyde Geronimi, Jack Kinney, Hamilton Luske","Roy Rogers, Trigger , Dennis Day, The Andrews ...",United States,1948,5/27/1948,TV-PG,76 min,Animation,...,Walt Disney Productions,None,Walt Disney,None,"['Eliot Daniel', 'Paul J. Smith', 'Ken Darby']","RKO Radio Pictures, Inc.",None,"['Donald Halliday', 'Thomas Scott']",None,12-Nov-19
3,National Treasure: Book of Secrets (2007),National Treasure: Book of Secrets,Jon Turteltaub,"Nicolas Cage, Jon Voight, Harvey Keitel, Ed Ha...",United States,2007,12/21/2007,PG,125 min,Action-Adventure,...,Walt Disney Pictures,"Jerry Bruckheimer Films, Junction Entertainmen...","['Jerry Bruckheimer', 'Jon Turteltaub']","['Characters', 'by', 'Jim Kouf', 'Oren Aviv', ...",Trevor Rabin,Walt Disney Studios Motion Pictures,"['John Schwartzman', 'Amir Mokri']","['William Goldenberg', 'David Rennie']","['Cormac Wibberley', 'Marianne Wibberley']",12-Nov-19
4,The Finest Hours (2016),The Finest Hours,Craig Gillespie,"Chris Pine, Casey Affleck, Ben Foster, Hollida...",United States,2016,1/25/2016,PG-13,119 min,Action-Adventure,...,Walt Disney Pictures,"Whitaker Entertainment, Red Hawk Entertainment","['Jim Whitaker', 'Dorothy Aufiero']","[""The Finest Hours: The True Story of the U.S....",Carter Burwell,"['Walt Disney Studios', 'Motion Pictures']",Javier Aguirresarobe,Tatiana S. Riegel,"['Scott Silver', 'Paul Tamasy', 'Eric Johnson']",6-Mar-20


In [52]:
%%bigquery
declare prompt_query STRING default "Based on the provided information about the movie and what you know about the movie, predict the release_date of the movie. For example, 10/5/1949, 3/4/2005, 10/30/2005, 7/24/1975, etc. Predict a value even if you are unsure. Return the output as json, include the item_id in the output as well";

create or replace table fp_disney_stg_ai.release_date1 as
  select *
  from ML.generate_text(
    model remote_models.gemini_pro,
    (
    select concat(prompt_query, to_json_string(json_object("item_id", dm.item_id, "title", dm.title,
"director", dm.director, "cast", dm.cast,
"country", dm.country, "release_year", dm.release_year,
"release_date", dm.release_date, "rating", dm.rating,
"movie_runtime", dm.movie_runtime, "genre1", dm.genre1,
"genre2", dm.genre2, "genre3", dm.genre3,
"description", dm.description, "budget", dm.budget,
"box_office", dm.box_office, "imdb", dm.imdb,
"metascore", dm.metascore, "rotten_tomatoes", dm.rotten_tomatoes,
"language", dm.language, "primary_production_company", dm.primary_production_company,
"additional_production_companies", dm.additional_production_companies, "produced_by", dm.produced_by,
"based_on", dm.based_on, "music_by", dm.music_by,
"distributed_by", dm.distributed_by, "cinematography", dm.cinematography,
"edited_by", dm.edited_by, "screenplay_by", dm.screenplay_by,
"date_added", dm.date_added))) as prompt
    from disney_stg.Disney_Movies as dm
    where release_date is null
    order by item_id
  ),
  struct(0 as temperature, 8192 as max_output_tokens, 0.0 as top_p, 1 as top_k, TRUE as flatten_json_output)
);


Query is running:   0%|          |

""


In [53]:
%%bigquery
select ml_generate_text_llm_result
from fp_disney_stg_ai.release_date1

Query is running:   0%|          |

Downloading:   0%|          |

,ml_generate_text_llm_result
0,"```json\n{""additional_production_companies"":nu..."
1,"```json\n{""additional_production_companies"":nu..."
2,"```json\n{""additional_production_companies"":nu..."
3,"```json\n{""additional_production_companies"":nu..."
4,"```json\n{""additional_production_companies"":nu..."
...,...
771,"```json\n{""additional_production_companies"":nu..."
772,"```json\n{""additional_production_companies"":nu..."
773,"```json\n{""additional_production_companies"":nu..."
774,"```json\n{""additional_production_companies"":nu..."


In [54]:
%%bigquery
select json_value(ml_generate_text_llm_result, '$.item_id') as item_id,
  json_value(ml_generate_text_llm_result, '$.release_date') as release_date
from fp_disney_stg_ai.release_date1

Query is running:   0%|          |

Downloading:   0%|          |

,item_id,release_date
0,None,None
1,None,None
2,None,None
3,None,None
4,None,None
...,...,...
771,None,None
772,None,None
773,None,None
774,None,None


In [55]:
%%bigquery
update disney_stg.Disney_Movies
  set data_source = 'release_date_ai' where release_date is null

Query is running:   0%|          |

""


In [56]:
%%bigquery
update disney_stg.Disney_Movies set release_date =
  (select json_value(ml_generate_text_llm_result, '$.release_date')
   from fp_disney_stg_ai.release_date1
   where item_id = json_value(ml_generate_text_llm_result, '$.item_id'))
where release_date is NULL

Query is running:   0%|          |

""


In [57]:
%%bigquery
select * except(data_source, load_time)
from disney_stg.Disney_Movies
where release_date is not null

Query is running:   0%|          |

Downloading:   0%|          |

,item_id,title,director,cast,country,release_year,release_date,rating,movie_runtime,genre1,...,primary_production_company,additional_production_companies,produced_by,based_on,music_by,distributed_by,cinematography,edited_by,screenplay_by,date_added
0,National Treasure (2004),National Treasure,Jon Turteltaub,"Nicolas Cage, Jon Voight, Harvey Keitel, Diane...",United States,2004,11/19/2004,PG,132 min,Action-Adventure,...,Walt Disney Pictures,"Jerry Bruckheimer Films, Junction Entertainmen...","['Jerry Bruckheimer', 'Jon Turteltaub']",None,Trevor Rabin,Buena Vista Pictures,Caleb Deschanel,William Goldenberg,"['Jim Kouf', 'Cormac Wibberley', 'Marianne Wib...",30-Apr-20
1,Tall Tale (1995),Tall Tale,Jeremiah Chechik,"Scott Glenn, Oliver Platt, Nick Stahl, Stephen...",United States,1995,3/24/1995,PG,97 min,Action-Adventure,...,Walt Disney Pictures,Caravan Pictures,"['Roger Birnbaum', 'Joe Roth']",None,Randy Edelman,Buena Vista Pictures,Janusz Kamiński,Richard Chew,None,12-Nov-19
2,Melody Time (1948),Melody Time,"Clyde Geronimi, Jack Kinney, Hamilton Luske","Roy Rogers, Trigger , Dennis Day, The Andrews ...",United States,1948,5/27/1948,TV-PG,76 min,Animation,...,Walt Disney Productions,None,Walt Disney,None,"['Eliot Daniel', 'Paul J. Smith', 'Ken Darby']","RKO Radio Pictures, Inc.",None,"['Donald Halliday', 'Thomas Scott']",None,12-Nov-19
3,National Treasure: Book of Secrets (2007),National Treasure: Book of Secrets,Jon Turteltaub,"Nicolas Cage, Jon Voight, Harvey Keitel, Ed Ha...",United States,2007,12/21/2007,PG,125 min,Action-Adventure,...,Walt Disney Pictures,"Jerry Bruckheimer Films, Junction Entertainmen...","['Jerry Bruckheimer', 'Jon Turteltaub']","['Characters', 'by', 'Jim Kouf', 'Oren Aviv', ...",Trevor Rabin,Walt Disney Studios Motion Pictures,"['John Schwartzman', 'Amir Mokri']","['William Goldenberg', 'David Rennie']","['Cormac Wibberley', 'Marianne Wibberley']",12-Nov-19
4,The Finest Hours (2016),The Finest Hours,Craig Gillespie,"Chris Pine, Casey Affleck, Ben Foster, Hollida...",United States,2016,1/25/2016,PG-13,119 min,Action-Adventure,...,Walt Disney Pictures,"Whitaker Entertainment, Red Hawk Entertainment","['Jim Whitaker', 'Dorothy Aufiero']","[""The Finest Hours: The True Story of the U.S....",Carter Burwell,"['Walt Disney Studios', 'Motion Pictures']",Javier Aguirresarobe,Tatiana S. Riegel,"['Scott Silver', 'Paul Tamasy', 'Eric Johnson']",6-Mar-20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,102 Dalmatians (2000),102 Dalmatians,Kevin Lima,"Glenn Close, Ioan Gruffudd, Alice Evans, Tim M...","United States, United Kingdom",2000,11/22/2000,G,104 min,Action-Adventure,...,Walt Disney Pictures,"Cruella Productions, Kanzaman S.A.M. Films",Edward S. Feldman,None,David Newman,Buena Vista Pictures,Adrian Biddle,Gregory Perler,"['Kristen Buckley', 'Brian Regan', 'Bob Tzudik...",12-Nov-19
272,Operation Dumbo Drop (1995),Operation Dumbo Drop,Simon Wincer,"Danny Glover, Ray Liotta, Denis Leary, Doug Do...",United States,1995,7/28/1995,PG,109 min,Action-Adventure,...,Walt Disney Pictures,"Interscope Communications, PolyGram Filmed Ent...","['Diane Nabatoff', 'David Madden']",None,David Newman,Buena Vista Pictures,Russell Boyd,O. Nicholas Brown,"['Gene Quintano', 'Jim Kouf']",12-Nov-19
273,The Young Black Stallion (2003),The Young Black Stallion,Simon Wincer,"Richard Romanus, Biana Tamimi, Patrick Elyas, ...",United States,2003,12/25/2003,G,51 min,Action-Adventure,...,Walt Disney Pictures,The Kennedy/Marshall Company,"['Fred Roos', 'Frank Marshall']","['Walter Farley', 'Steven Farley']",William Ross,Buena Vista Pictures,Reed Smoot,"['Bud Smith', 'Terry Blythe', 'M. Scott Smith']",Jeanne Rosenberg,12-Nov-19
274,Benji the Hunted (1987),Benji the Hunted,Joe Camp,"Red Steagall, Frank Inn, Nancy Francis, Mike F...",United States,1987,6/5/1987,G,89 min,Action-Adventure,...,Walt Disney Pictures,"Mulberry Square Productions, Silver Screen Par...",Ben Vaughn,None,"['Betty Box', 'Euel Box']"

In [58]:
%%bigquery
select count(*) from disney_stg.Disney_Movies
where release_date is NULL;

Query is running:   0%|          |

Downloading:   0%|          |

,f0_
0,776


#### budget

In [59]:
%%bigquery
select * except (data_source, load_time)
from disney_stg.Disney_Movies
where budget is not null
limit 5

Query is running:   0%|          |

Downloading:   0%|          |

,item_id,title,director,cast,country,release_year,release_date,rating,movie_runtime,genre1,...,primary_production_company,additional_production_companies,produced_by,based_on,music_by,distributed_by,cinematography,edited_by,screenplay_by,date_added
0,National Treasure (2004),National Treasure,Jon Turteltaub,"Nicolas Cage, Jon Voight, Harvey Keitel, Diane...",United States,2004,11/19/2004,PG,132 min,Action-Adventure,...,Walt Disney Pictures,"Jerry Bruckheimer Films, Junction Entertainmen...","['Jerry Bruckheimer', 'Jon Turteltaub']",None,Trevor Rabin,Buena Vista Pictures,Caleb Deschanel,William Goldenberg,"['Jim Kouf', 'Cormac Wibberley', 'Marianne Wib...",30-Apr-20
1,Tall Tale (1995),Tall Tale,Jeremiah Chechik,"Scott Glenn, Oliver Platt, Nick Stahl, Stephen...",United States,1995,3/24/1995,PG,97 min,Action-Adventure,...,Walt Disney Pictures,Caravan Pictures,"['Roger Birnbaum', 'Joe Roth']",None,Randy Edelman,Buena Vista Pictures,Janusz Kamiński,Richard Chew,None,12-Nov-19
2,Melody Time (1948),Melody Time,"Clyde Geronimi, Jack Kinney, Hamilton Luske","Roy Rogers, Trigger , Dennis Day, The Andrews ...",United States,1948,5/27/1948,TV-PG,76 min,Animation,...,Walt Disney Productions,None,Walt Disney,None,"['Eliot Daniel', 'Paul J. Smith', 'Ken Darby']","RKO Radio Pictures, Inc.",None,"['Donald Halliday', 'Thomas Scott']",None,12-Nov-19
3,National Treasure: Book of Secrets (2007),National Treasure: Book of Secrets,Jon Turteltaub,"Nicolas Cage, Jon Voight, Harvey Keitel, Ed Ha...",United States,2007,12/21/2007,PG,125 min,Action-Adventure,...,Walt Disney Pictures,"Jerry Bruckheimer Films, Junction Entertainmen...","['Jerry Bruckheimer', 'Jon Turteltaub']","['Characters', 'by', 'Jim Kouf', 'Oren Aviv', ...",Trevor Rabin,Walt Disney Studios Motion Pictures,"['John Schwartzman', 'Amir Mokri']","['William Goldenberg', 'David Rennie']","['Cormac Wibberley', 'Marianne Wibberley']",12-Nov-19
4,The Finest Hours (2016),The Finest Hours,Craig Gillespie,"Chris Pine, Casey Affleck, Ben Foster, Hollida...",United States,2016,1/25/2016,PG-13,119 min,Action-Adventure,...,Walt Disney Pictures,"Whitaker Entertainment, Red Hawk Entertainment","['Jim Whitaker', 'Dorothy Aufiero']","[""The Finest Hours: The True Story of the U.S....",Carter Burwell,"['Walt Disney Studios', 'Motion Pictures']",Javier Aguirresarobe,Tatiana S. Riegel,"['Scott Silver', 'Paul Tamasy', 'Eric Johnson']",6-Mar-20


In [60]:
%%bigquery
declare prompt_query STRING default "Based on the provided information about the movie and what you know about the movie, predict the budget of the movie. For example, 56000000, 150000000, 14000000, 115000000, etc. Predict a value even if you are unsure. Return the output as json, include the item_id in the output as well";

create or replace table fp_disney_stg_ai.budget1 as
  select *
  from ML.generate_text(
    model remote_models.gemini_pro,
    (
    select concat(prompt_query, to_json_string(json_object("item_id", dm.item_id, "title", dm.title,
"director", dm.director, "cast", dm.cast,
"country", dm.country, "release_year", dm.release_year,
"release_date", dm.release_date, "rating", dm.rating,
"movie_runtime", dm.movie_runtime, "genre1", dm.genre1,
"genre2", dm.genre2, "genre3", dm.genre3,
"description", dm.description, "budget", dm.budget,
"box_office", dm.box_office, "imdb", dm.imdb,
"metascore", dm.metascore, "rotten_tomatoes", dm.rotten_tomatoes,
"language", dm.language, "primary_production_company", dm.primary_production_company,
"additional_production_companies", dm.additional_production_companies, "produced_by", dm.produced_by,
"based_on", dm.based_on, "music_by", dm.music_by,
"distributed_by", dm.distributed_by, "cinematography", dm.cinematography,
"edited_by", dm.edited_by, "screenplay_by", dm.screenplay_by,
"date_added", dm.date_added))) as prompt
    from disney_stg.Disney_Movies as dm
    where budget is null
    order by item_id
  ),
  struct(0 as temperature, 8192 as max_output_tokens, 0.0 as top_p, 1 as top_k, TRUE as flatten_json_output)
);


Query is running:   0%|          |

""


In [61]:
%%bigquery
select ml_generate_text_llm_result
from fp_disney_stg_ai.budget1

Query is running:   0%|          |

Downloading:   0%|          |

,ml_generate_text_llm_result
0,"```json\n{""additional_production_companies"":nu..."
1,"```json\n{""additional_production_companies"":nu..."
2,"```json\n{""additional_production_companies"":nu..."
3,"```json\n{""additional_production_companies"":nu..."
4,"```json\n{""additional_production_companies"":nu..."
...,...
847,"```json\n{""additional_production_companies"":nu..."
848,"```json\n{""additional_production_companies"":nu..."
849,"```json\n{""additional_production_companies"":nu..."
850,"```json\n{""additional_production_companies"":nu..."


In [62]:
%%bigquery
select json_value(ml_generate_text_llm_result, '$.item_id') as item_id,
  json_value(ml_generate_text_llm_result, '$.budget') as budget
from fp_disney_stg_ai.budget1

Query is running:   0%|          |

Downloading:   0%|          |

,item_id,budget
0,None,None
1,None,None
2,None,None
3,None,None
4,None,None
...,...,...
847,None,None
848,None,None
849,None,None
850,None,None


In [63]:
%%bigquery
update disney_stg.Disney_Movies
  set data_source = 'budget_ai' where budget is null

Query is running:   0%|          |

""


In [64]:
%%bigquery
update disney_stg.Disney_Movies set budget =
  (select json_value(ml_generate_text_llm_result, '$.budget')
   from fp_disney_stg_ai.budget1
   where item_id = json_value(ml_generate_text_llm_result, '$.item_id'))
where budget is NULL

Query is running:   0%|          |

""


In [65]:
%%bigquery
select * except(data_source, load_time)
from disney_stg.Disney_Movies
where budget is not null

Query is running:   0%|          |

Downloading:   0%|          |

,item_id,title,director,cast,country,release_year,release_date,rating,movie_runtime,genre1,...,primary_production_company,additional_production_companies,produced_by,based_on,music_by,distributed_by,cinematography,edited_by,screenplay_by,date_added
0,Tall Tale (1995),Tall Tale,Jeremiah Chechik,"Scott Glenn, Oliver Platt, Nick Stahl, Stephen...",United States,1995,3/24/1995,PG,97 min,Action-Adventure,...,Walt Disney Pictures,Caravan Pictures,"['Roger Birnbaum', 'Joe Roth']",None,Randy Edelman,Buena Vista Pictures,Janusz Kamiński,Richard Chew,None,12-Nov-19
1,National Treasure (2004),National Treasure,Jon Turteltaub,"Nicolas Cage, Jon Voight, Harvey Keitel, Diane...",United States,2004,11/19/2004,PG,132 min,Action-Adventure,...,Walt Disney Pictures,"Jerry Bruckheimer Films, Junction Entertainmen...","['Jerry Bruckheimer', 'Jon Turteltaub']",None,Trevor Rabin,Buena Vista Pictures,Caleb Deschanel,William Goldenberg,"['Jim Kouf', 'Cormac Wibberley', 'Marianne Wib...",30-Apr-20
2,Melody Time (1948),Melody Time,"Clyde Geronimi, Jack Kinney, Hamilton Luske","Roy Rogers, Trigger , Dennis Day, The Andrews ...",United States,1948,5/27/1948,TV-PG,76 min,Animation,...,Walt Disney Productions,None,Walt Disney,None,"['Eliot Daniel', 'Paul J. Smith', 'Ken Darby']","RKO Radio Pictures, Inc.",None,"['Donald Halliday', 'Thomas Scott']",None,12-Nov-19
3,National Treasure: Book of Secrets (2007),National Treasure: Book of Secrets,Jon Turteltaub,"Nicolas Cage, Jon Voight, Harvey Keitel, Ed Ha...",United States,2007,12/21/2007,PG,125 min,Action-Adventure,...,Walt Disney Pictures,"Jerry Bruckheimer Films, Junction Entertainmen...","['Jerry Bruckheimer', 'Jon Turteltaub']","['Characters', 'by', 'Jim Kouf', 'Oren Aviv', ...",Trevor Rabin,Walt Disney Studios Motion Pictures,"['John Schwartzman', 'Amir Mokri']","['William Goldenberg', 'David Rennie']","['Cormac Wibberley', 'Marianne Wibberley']",12-Nov-19
4,The Finest Hours (2016),The Finest Hours,Craig Gillespie,"Chris Pine, Casey Affleck, Ben Foster, Hollida...",United States,2016,1/25/2016,PG-13,119 min,Action-Adventure,...,Walt Disney Pictures,"Whitaker Entertainment, Red Hawk Entertainment","['Jim Whitaker', 'Dorothy Aufiero']","[""The Finest Hours: The True Story of the U.S....",Carter Burwell,"['Walt Disney Studios', 'Motion Pictures']",Javier Aguirresarobe,Tatiana S. Riegel,"['Scott Silver', 'Paul Tamasy', 'Eric Johnson']",6-Mar-20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,Cheetah (1989),Cheetah,Jeff Blyth,"Keith Coogan, Lucy Deakins, Timothy Landfield,...",United States,1989,8/18/1989,G,84 min,Action-Adventure,...,Walt Disney Pictures,Silver Screen Partners III,"['Roy E. Disney', 'Robert Halmi, Sr.']","['The Cheetahs', 'by', 'Alan Caillou']",Bruce Rowland,Buena Vista Pictures,Thomas Burstyn,Eric Albertson,"['Erik Tarloff', 'John Cotter', 'Griff Du Rhone']",12-Nov-19
197,Eight Below (2006),Eight Below,Frank Marshall,"Paul Walker, Bruce Greenwood, Moon Bloodgood, ...",United States,2006,2/17/2006,PG,122 min,Action-Adventure,...,Walt Disney Pictures,"Spyglass Entertainment, Mandeville Films, The ...","['Patrick Crowley', 'David Hoberman']","['Toshirô Ishidô', 'Koreyoshi Kurahara', 'Tats...",Mark Isham,Buena Vista Pictures,Don Burgess,Christopher Rouse,David DiGillio,12-Nov-19
198,White Fang (1991),White Fang,Randal Kleiser,"Klaus Brandauer, Ethan Hawke, Seymour Cassel, ...",United States,1991,1/18/1991,PG,109 min,Action-Adventure,...,Walt Disney Pictures,Silver Screen Partners IV,Markay Powell,"['White Fang', 'by', 'Jack London']","['Basil Poledouris', 'Hans Zimmer', '(addition...",Buena Vista Pictures,Tony Pierce-Roberts,Lisa Day,"['Jeanne Rosenberg', 'Nick Thiel', 'David Fall...",12-Nov-19
199,102 Dalmatians (2000),102 Dalmatians,Kevin Lima,"Glenn Close, Ioan Gruffudd, Alice Evans, Tim M...","United States, United Kingdom",2000,11/22/2000,G,104 min,Action-Adventure,...,Walt Disney Pictures,"Cruella Productions, Kanzaman S.A.M. Films",Edward S. Fel

In [66]:
%%bigquery
select count(*) from disney_stg.Disney_Movies
where budget is NULL;

Query is running:   0%|          |

Downloading:   0%|          |

,f0_
0,851


#### box_office

In [67]:
%%bigquery
select * except (data_source, load_time)
from disney_stg.Disney_Movies
where box_office is not null
limit 5

Query is running:   0%|          |

Downloading:   0%|          |

,item_id,title,director,cast,country,release_year,release_date,rating,movie_runtime,genre1,...,primary_production_company,additional_production_companies,produced_by,based_on,music_by,distributed_by,cinematography,edited_by,screenplay_by,date_added
0,Tall Tale (1995),Tall Tale,Jeremiah Chechik,"Scott Glenn, Oliver Platt, Nick Stahl, Stephen...",United States,1995,3/24/1995,PG,97 min,Action-Adventure,...,Walt Disney Pictures,Caravan Pictures,"['Roger Birnbaum', 'Joe Roth']",None,Randy Edelman,Buena Vista Pictures,Janusz Kamiński,Richard Chew,None,12-Nov-19
1,National Treasure (2004),National Treasure,Jon Turteltaub,"Nicolas Cage, Jon Voight, Harvey Keitel, Diane...",United States,2004,11/19/2004,PG,132 min,Action-Adventure,...,Walt Disney Pictures,"Jerry Bruckheimer Films, Junction Entertainmen...","['Jerry Bruckheimer', 'Jon Turteltaub']",None,Trevor Rabin,Buena Vista Pictures,Caleb Deschanel,William Goldenberg,"['Jim Kouf', 'Cormac Wibberley', 'Marianne Wib...",30-Apr-20
2,Melody Time (1948),Melody Time,"Clyde Geronimi, Jack Kinney, Hamilton Luske","Roy Rogers, Trigger , Dennis Day, The Andrews ...",United States,1948,5/27/1948,TV-PG,76 min,Animation,...,Walt Disney Productions,None,Walt Disney,None,"['Eliot Daniel', 'Paul J. Smith', 'Ken Darby']","RKO Radio Pictures, Inc.",None,"['Donald Halliday', 'Thomas Scott']",None,12-Nov-19
3,National Treasure: Book of Secrets (2007),National Treasure: Book of Secrets,Jon Turteltaub,"Nicolas Cage, Jon Voight, Harvey Keitel, Ed Ha...",United States,2007,12/21/2007,PG,125 min,Action-Adventure,...,Walt Disney Pictures,"Jerry Bruckheimer Films, Junction Entertainmen...","['Jerry Bruckheimer', 'Jon Turteltaub']","['Characters', 'by', 'Jim Kouf', 'Oren Aviv', ...",Trevor Rabin,Walt Disney Studios Motion Pictures,"['John Schwartzman', 'Amir Mokri']","['William Goldenberg', 'David Rennie']","['Cormac Wibberley', 'Marianne Wibberley']",12-Nov-19
4,The Finest Hours (2016),The Finest Hours,Craig Gillespie,"Chris Pine, Casey Affleck, Ben Foster, Hollida...",United States,2016,1/25/2016,PG-13,119 min,Action-Adventure,...,Walt Disney Pictures,"Whitaker Entertainment, Red Hawk Entertainment","['Jim Whitaker', 'Dorothy Aufiero']","[""The Finest Hours: The True Story of the U.S....",Carter Burwell,"['Walt Disney Studios', 'Motion Pictures']",Javier Aguirresarobe,Tatiana S. Riegel,"['Scott Silver', 'Paul Tamasy', 'Eric Johnson']",6-Mar-20


In [68]:
%%bigquery
declare prompt_query STRING default "Based on the provided information about the movie and what you know about the movie, predict the box_office score of the movie. For example, 347500000, 11000000, 2560000, 459200000, etc. Predict a value even if you are unsure. Return the output as json, include the item_id in the output as well";

create or replace table fp_disney_stg_ai.box_office1 as
  select *
  from ML.generate_text(
    model remote_models.gemini_pro,
    (
    select concat(prompt_query, to_json_string(json_object("item_id", dm.item_id, "title", dm.title,
"director", dm.director, "cast", dm.cast,
"country", dm.country, "release_year", dm.release_year,
"release_date", dm.release_date, "rating", dm.rating,
"movie_runtime", dm.movie_runtime, "genre1", dm.genre1,
"genre2", dm.genre2, "genre3", dm.genre3,
"description", dm.description, "budget", dm.budget,
"box_office", dm.box_office, "imdb", dm.imdb,
"metascore", dm.metascore, "rotten_tomatoes", dm.rotten_tomatoes,
"language", dm.language, "primary_production_company", dm.primary_production_company,
"additional_production_companies", dm.additional_production_companies, "produced_by", dm.produced_by,
"based_on", dm.based_on, "music_by", dm.music_by,
"distributed_by", dm.distributed_by, "cinematography", dm.cinematography,
"edited_by", dm.edited_by, "screenplay_by", dm.screenplay_by,
"date_added", dm.date_added))) as prompt
    from disney_stg.Disney_Movies as dm
    where box_office is null
    order by item_id
  ),
  struct(0 as temperature, 8192 as max_output_tokens, 0.0 as top_p, 1 as top_k, TRUE as flatten_json_output)
);


Query is running:   0%|          |

""


In [69]:
%%bigquery
select ml_generate_text_llm_result
from fp_disney_stg_ai.box_office1

Query is running:   0%|          |

Downloading:   0%|          |

,ml_generate_text_llm_result
0,"```json\n{""additional_production_companies"":nu..."
1,"```json\n{""additional_production_companies"":nu..."
2,"```json\n{""additional_production_companies"":nu..."
3,"```json\n{""additional_production_companies"":nu..."
4,"```json\n{""additional_production_companies"":nu..."
...,...
806,"```json\n{""additional_production_companies"":nu..."
807,"```json\n{""additional_production_companies"":nu..."
808,"```json\n{""additional_production_companies"":nu..."
809,"```json\n{""additional_production_companies"":nu..."


In [70]:
%%bigquery
select json_value(ml_generate_text_llm_result, '$.item_id') as item_id,
  json_value(ml_generate_text_llm_result, '$.box_office') as box_office
from fp_disney_stg_ai.box_office1

Query is running:   0%|          |

Downloading:   0%|          |

,item_id,box_office
0,None,None
1,None,None
2,None,None
3,None,None
4,None,None
...,...,...
806,None,None
807,None,None
808,None,None
809,None,None


In [71]:
%%bigquery
update disney_stg.Disney_Movies
  set data_source = 'box_office_ai' where box_office is null

Query is running:   0%|          |

""


In [89]:
%%bigquery
UPDATE disney_stg.Disney_Movies
SET box_office = CAST(
    REPLACE(
        (SELECT json_value(ml_generate_text_llm_result, '$.box_office')
         FROM fp_disney_stg_ai.box_office1
         WHERE item_id = json_value(ml_generate_text_llm_result, '$.item_id')),
         ',', '') AS INT64
)
WHERE box_office IS NULL;


Query is running:   0%|          |

""


In [90]:
%%bigquery
select * except(data_source, load_time)
from disney_stg.Disney_Movies
where box_office is not null

Query is running:   0%|          |

Downloading:   0%|          |

,item_id,title,director,cast,country,release_year,release_date,rating,movie_runtime,genre1,...,primary_production_company,additional_production_companies,produced_by,based_on,music_by,distributed_by,cinematography,edited_by,screenplay_by,date_added
0,Tall Tale (1995),Tall Tale,Jeremiah Chechik,"Scott Glenn, Oliver Platt, Nick Stahl, Stephen...",United States,1995,3/24/1995,PG,97 min,Action-Adventure,...,Walt Disney Pictures,Caravan Pictures,"['Roger Birnbaum', 'Joe Roth']",None,Randy Edelman,Buena Vista Pictures,Janusz Kamiński,Richard Chew,None,12-Nov-19
1,National Treasure (2004),National Treasure,Jon Turteltaub,"Nicolas Cage, Jon Voight, Harvey Keitel, Diane...",United States,2004,11/19/2004,PG,132 min,Action-Adventure,...,Walt Disney Pictures,"Jerry Bruckheimer Films, Junction Entertainmen...","['Jerry Bruckheimer', 'Jon Turteltaub']",None,Trevor Rabin,Buena Vista Pictures,Caleb Deschanel,William Goldenberg,"['Jim Kouf', 'Cormac Wibberley', 'Marianne Wib...",30-Apr-20
2,Melody Time (1948),Melody Time,"Clyde Geronimi, Jack Kinney, Hamilton Luske","Roy Rogers, Trigger , Dennis Day, The Andrews ...",United States,1948,5/27/1948,TV-PG,76 min,Animation,...,Walt Disney Productions,None,Walt Disney,None,"['Eliot Daniel', 'Paul J. Smith', 'Ken Darby']","RKO Radio Pictures, Inc.",None,"['Donald Halliday', 'Thomas Scott']",None,12-Nov-19
3,National Treasure: Book of Secrets (2007),National Treasure: Book of Secrets,Jon Turteltaub,"Nicolas Cage, Jon Voight, Harvey Keitel, Ed Ha...",United States,2007,12/21/2007,PG,125 min,Action-Adventure,...,Walt Disney Pictures,"Jerry Bruckheimer Films, Junction Entertainmen...","['Jerry Bruckheimer', 'Jon Turteltaub']","['Characters', 'by', 'Jim Kouf', 'Oren Aviv', ...",Trevor Rabin,Walt Disney Studios Motion Pictures,"['John Schwartzman', 'Amir Mokri']","['William Goldenberg', 'David Rennie']","['Cormac Wibberley', 'Marianne Wibberley']",12-Nov-19
4,The Finest Hours (2016),The Finest Hours,Craig Gillespie,"Chris Pine, Casey Affleck, Ben Foster, Hollida...",United States,2016,1/25/2016,PG-13,119 min,Action-Adventure,...,Walt Disney Pictures,"Whitaker Entertainment, Red Hawk Entertainment","['Jim Whitaker', 'Dorothy Aufiero']","[""The Finest Hours: The True Story of the U.S....",Carter Burwell,"['Walt Disney Studios', 'Motion Pictures']",Javier Aguirresarobe,Tatiana S. Riegel,"['Scott Silver', 'Paul Tamasy', 'Eric Johnson']",6-Mar-20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,Eight Below (2006),Eight Below,Frank Marshall,"Paul Walker, Bruce Greenwood, Moon Bloodgood, ...",United States,2006,2/17/2006,PG,122 min,Action-Adventure,...,Walt Disney Pictures,"Spyglass Entertainment, Mandeville Films, The ...","['Patrick Crowley', 'David Hoberman']","['Toshirô Ishidô', 'Koreyoshi Kurahara', 'Tats...",Mark Isham,Buena Vista Pictures,Don Burgess,Christopher Rouse,David DiGillio,12-Nov-19
239,White Fang (1991),White Fang,Randal Kleiser,"Klaus Brandauer, Ethan Hawke, Seymour Cassel, ...",United States,1991,1/18/1991,PG,109 min,Action-Adventure,...,Walt Disney Pictures,Silver Screen Partners IV,Markay Powell,"['White Fang', 'by', 'Jack London']","['Basil Poledouris', 'Hans Zimmer', '(addition...",Buena Vista Pictures,Tony Pierce-Roberts,Lisa Day,"['Jeanne Rosenberg', 'Nick Thiel', 'David Fall...",12-Nov-19
240,102 Dalmatians (2000),102 Dalmatians,Kevin Lima,"Glenn Close, Ioan Gruffudd, Alice Evans, Tim M...","United States, United Kingdom",2000,11/22/2000,G,104 min,Action-Adventure,...,Walt Disney Pictures,"Cruella Productions, Kanzaman S.A.M. Films",Edward S. Feldman,None,David Newman,Buena Vista Pictures,Adrian Biddle,Gregory Perler,"['Kristen Buckley', 'Brian Regan', 'Bob Tzudik...",12-Nov-19
241,Benji the Hunted (1987),Benji the Hunted,Joe Camp,"Red Steagall, Frank Inn, Nancy Francis, Mike F...",United States,1987,6/5/1987,G,89 min,Action-Adventure,...,Walt Disney Pictures,"Mulberry Square Productions, Silver Screen Par...",Ben Vaughn,None,"['Betty Box', 'Eu

In [91]:
%%bigquery
select count(*) from disney_stg.Disney_Movies
where box_office is NULL;

Query is running:   0%|          |

Downloading:   0%|          |

,f0_
0,809
